In [1]:
import numpy as np

In [2]:
import requests
import os

In [3]:
import cv2
from PIL import Image

In [4]:
import matplotlib.pyplot as plt

In [ ]:
os.chdir('Documents/Projects/ImageRec')

In [5]:
def img_save(query_term, offset):
    '''
    saves and processes images for specified query, creates folder in directory if there isnt one
    '''
    
    API_KEY = os.getenv('bing_search_api_key')
    URL = "https://api.cognitive.microsoft.com/bing/v7.0/images/search"
    USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)' + \
                 'Chrome/80.0.3987.87 Safari/537.36'
    
    # set path with query term as file name
    path = os.getcwd().replace('\\', '/') + '/' + str(query_term) + '/'
    
    if not os.path.exists(query_term):
        # if the directory does not exist, make one
        os.mkdir(query_term)
        
    headers = {"Ocp-Apim-Subscription-Key" : API_KEY}
    params = {"q": query_term,
              "count": 150, 
              "offset": offset * 150}

    # Search Bing for images
    search = requests.get(URL, headers=headers, params=params)
    results = search.json()
    
    print('TOTAL ESTIMATED MATCHES: ' + str(results['totalEstimatedMatches']))

    # Save all of the resulting images from each page
    num = offset * 150
    for value in results['value']:

        print(str(num) + ' ' + value['contentUrl'], end='\r')
        
        try:
            image = requests.get(value["contentUrl"], timeout=30, headers={'User-Agent': USER_AGENT})
            
        except(requests.ConnectionError):
            print(str(num) + ' BAD CONNECTION', end='\r')
            continue
            
        except(requests.ReadTimeout):
            print(str(num) + ' TIMEOUT', end='\r')
            continue
        
        # Check the status of the request - If the image does not exist we will skip it
        try:
            image.raise_for_status()
            
            file = open(path + query_term + '_' + str(offset) + '_' + str(num) + '.png', 'wb')
            file.write(image.content)
            file.close()
            
        except(requests.HTTPError):
            print(str(num) + ' NOT FOUND', end='\r')
            
        num += 1

In [6]:
def img_format(folder):
    '''
    read in all images in given folder, format them, and put them into a master array
    return x_data array, y_data array
    '''
    
    x_data = np.array([])
    Y_data = np.array([])
    
    # Iterate through each file in the specified folder
    for file in os.listdir(folder):
        
        print(file, end='\r')
        
        # Read in the image
        try:
            img = np.array(Image.open(folder + '/' + file))
        except(IOError):
            print('Image not found')
            continue
        
        # If the image is greyscale, discard it
        if len(img.shape) == 2:
            continue
        
        # If the image is 4 channel (RGBA), convert to 3 channel (RGB)
        if (len(img.shape) > 2) & (img.shape[2] == 4):
            img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
        
        # Identify which sides need to be padded and by how much, to make the image square
        short = np.argmin(img.shape[:2])
        diff_1 = int(np.ceil(abs(img.shape[1] - img.shape[0])/2))
        diff_2 = int(np.floor(abs(img.shape[1] - img.shape[0])/2))
        
        # Set the desired padding on the short side, and apply
        width = [[0, 0], [0, 0], [0, 0]]
        width[short] = [diff_1, diff_2]
        img = np.pad(img, pad_width=width, mode='constant')
        
        # Resize square image to 100x100
        img = cv2.resize(img, (100, 100))
        
        # Reshape array to be appended to x_data array
        img = img.reshape(1, 100, 100, 3)
        
        # Put the formatted arrays into a master array of training data
        if np.array_equal(x_data, np.array([])):
            # This is the first one, start the array
            x_data = img
            
        else:
            # Append to full array
            x_data = np.concatenate((x_data, img), axis=0)
        
        # The class label will be same as the name of the folder
        Y_data = np.append(Y_data, folder)
    
    return x_data, Y_data

#### Save the images from search queries

In [9]:
img_save('cat', 0)
img_save('cat', 1)
img_save('cat', 2)
img_save('cat', 3)

TOTAL ESTIMATED MATCHES: 471


In [11]:
img_save('dog', 0)
img_save('dog', 1)
img_save('dog', 2)
img_save('dog', 3)

TOTAL ESTIMATED MATCHES: 477
TOTAL ESTIMATED MATCHES: 477D1xgR3Jvwo/hqdefault.jpg.jpg01/cho-gam-xuong-tapchichomeo.com_.jpgo-nu/33108-dog-flowers.JPGdu_after_Kukur_Puja.jpg120807x9Us_nGv0f-jKTG8_UinfF707bkJFz9bVwWS7M=
TOTAL ESTIMATED MATCHES: 482/_UTzIAi3vBcY/TQ2b3Y9-WgI/AAAAAAAAA0E/bKDw6b_lY9Q/s1600/WebHund62010.jpgl.jpgdrodysplasia-skeletal-dwarfism.jpg0728120807jpgackgrounds-animal.jpgdadi.jpg
TOTAL ESTIMATED MATCHES: 482g.uk/dogimages/1237659_adele_20200222082606_adele-g-hound-1-(2)_800.jpgjpggpgcover-image-732x412.jpgSwimming-Preserver-Summer-Swimwear_ce16de60-4141-4f25-bb67-f37921ece882_1200x1200.jpg?v=1582154787-Dog_relief_from_me%CA%BBae_I%CA%BBipona%2C_Puama%CA%BBu_Village%2C_Hiva_Oa%2C_Marquesas_Islands%2C_photograph_by_Moth_Clark%2C_2009_%28levels_adjusted%29.jpg


In [12]:
img_save('snake', 0)
img_save('snake', 1)
img_save('snake', 2)
img_save('snake', 3)

TOTAL ESTIMATED MATCHES: 666
TOTAL ESTIMATED MATCHES: 666661kKremyk/maxresdefault.jpg11/10/img_3668_crop.jpg181-brown_tree_snake_boiga_irregularis_-spl.jpgm1VO347jJGrJ3Na7Ed4.jpeg?imwidth=450-elapidae-emydidae-scaled-reptile-1386581.jpg
TOTAL ESTIMATED MATCHES: 6661.1449254989!/fileImage/httpImage/image.png_gen/derivatives/16x9_1180/rattlesnake-with-fungal-infection.png1QiT4Uvmb-TOYSvIjNj5sFqJg0igJb7b-_Q=rix_helvetica%29_playing_dead_%2814178349634%29.jpg-head-photograph-snake-rattlesnake-up-still-sony-detail-manual-dof-body-alpha-snakes-short-medium-garter-m42-jupiter-serpent-slither-telephoto-a330-jupiter37a-37a-basking-organism-viper-grass-snake-garter-snake-colubridae-sidewinder-scaled-reptile-lacertidae-hognose-snake-kingsnake-terrestrial-animal-527141.jpg
TOTAL ESTIMATED MATCHES: 666/wp-content/uploads/2015/02/Titanoboa-11-1024x760.jpglithering-surprise-australian-woman-finds-snake-on-her-christmas-tree/_jcr_content/par/featured-media/media-0.img.jpg/0/0/1482165352895.jpg?ve=1er_

In [13]:
img_save('penguin', 0)
img_save('penguin', 1)
img_save('penguin', 2)
img_save('penguin', 3)

TOTAL ESTIMATED MATCHES: 969
TOTAL ESTIMATED MATCHES: 969/-Uskp0bKYGok/TmxNx-8_yOI/AAAAAAAAA4U/bVBXY131rIc/s1600/penguin_3.jpgIMG_7756_352085.jpgjpg807a63557a-2060x1236.jpeg?w=1200&q=55&auto=format&usm=12&fit=max&s=349a333b7929f9c0c9343ae0f5720445Rakiura+penguins+beautiful+amazing+animal+pictures.jpg
TOTAL ESTIMATED MATCHES: 969ca.com/blog/advocacy/wp-content/uploads/emperor-penguin.jpg0-p-k-no-nu/Emperor+Penguin1.jpgg2%2C2877%2C1922&q=45&auto=format&w=496&fit=clipgs=503cdb6b67cad5d3fabf3953d40fa74cIsland+penguins+Rakiura+penguins+beautiful+amazing+animal+pictures.jpg
TOTAL ESTIMATED MATCHES: 969.org.nz/sites/all/files/4701112X2A7636%20-%20Copy%20ps%20ed%20crop%20bol.jpgx-LittlePenguin_PerthZoo_SMCSept05.jpg-3840x2400.jpgctica-polar-regions.jpgLBrd-01-09-2016-Daily-1-D002--2016-01-08-IMG-7-Little-Blue-Pengui-1-1-GED3JGFE-L740593295-IMG-7-Little-Blue-Pengui-1-1-GED3JGFE.jpg


In [14]:
img_save('dolphin', 0)
img_save('dolphin', 1)
img_save('dolphin', 2)
img_save('dolphin', 3)

TOTAL ESTIMATED MATCHES: 960
TOTAL ESTIMATED MATCHES: 958.uk/content/dam/science/2017/11/03/shutterstock_736129444_trans_NvBQzQNjv4BqZgEkZX3M936N5BQK4Va8RUbgHFEZVI1Pljic_pW9c90.jpg?imwidth=4500fb0026652224.jpgdf78c3c4f806171.jpgon-154724035-59ce93949abed50011352530.jpg
TOTAL ESTIMATED MATCHES: 959com/wp-content/gallery/swim-with-dolphins-article/Dolphind.jpg422282248365.adapt.1900.1.jpgl/-/media/2015/01/06/Phoenix/Phoenix/635561620095703087-4-14---Dolphin.jpgHusbandry-20-Edit.jpgy,P3D420,P26q,P3D85,P26i,P3D,P2Fdocuments,P2Ffeatures,P2Fimages,P2F25872_dolphins.jpg.pagespeed.ic.6WsmFtgz98.jpg
TOTAL ESTIMATED MATCHES: 959st.com/resizer/fecjxmtSXJZURawZqfhTOkLSR8I=/1484x0/arc-anglerfish-washpost-prod-washpost.s3.amazonaws.com/public/CTAPM5XSVFBVVPNMHE4R526SJI.jpgapt.1900.1.jpgphin.jpg


#### Format the images and save to arrays

In [9]:
x_cat, Y_cat = img_format('cat')
x_dog, Y_dog = img_format('dog')
x_snake, Y_snake = img_format('snake')
x_penguin, Y_penguin = img_format('penguin')
x_dolphin, Y_dolphin = img_format('dolphin')

In [14]:
x_train = np.concatenate((x_cat, x_dog, x_snake, x_penguin, x_dolphin))
Y_train = np.concatenate((Y_cat, Y_dog, Y_snake, Y_penguin, Y_dolphin))

In [16]:
Y_train[Y_train == 'cat'] = 0
Y_train[Y_train == 'dog'] = 1
Y_train[Y_train == 'snake'] = 2
Y_train[Y_train == 'penguin'] = 3
Y_train[Y_train == 'dolphin'] = 4

In [17]:
Y_train = Y_train.astype(np.int)

#### Train Neural Network

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D

In [19]:
nn_model = Sequential()

In [20]:
nn_model.add(Dense(16, input_shape=x_train[0].shape, activation='relu'))
nn_model.add(Dense(16,  activation='relu'))
nn_model.add(Flatten())
nn_model.add(Dense(len(np.unique(Y_train)),  activation='softmax'))

In [21]:
nn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100, 100, 16)      64        
_________________________________________________________________
dense_1 (Dense)              (None, 100, 100, 16)      272       
_________________________________________________________________
flatten (Flatten)            (None, 160000)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 800005    
Total params: 800,341
Trainable params: 800,341
Non-trainable params: 0
_________________________________________________________________


In [22]:
nn_model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

nn_model.fit(x=x_train, y=Y_train, epochs=10)

Train on 2008 samples
Epoch 1/10
2008/2008 [==============================] - 9s 5ms/sample - loss: 161.1009 - accuracy: 0.4841
Epoch 2/10
2008/2008 [==============================] - 8s 4ms/sample - loss: 24.2865 - accuracy: 0.6121
Epoch 3/10
2008/2008 [==============================] - 8s 4ms/sample - loss: 6.3226 - accuracy: 0.6997
Epoch 4/10
2008/2008 [==============================] - 8s 4ms/sample - loss: 2.3835 - accuracy: 0.7704
Epoch 5/10
2008/2008 [==============================] - 8s 4ms/sample - loss: 1.0731 - accuracy: 0.8292
Epoch 6/10
2008/2008 [==============================] - 8s 4ms/sample - loss: 0.5216 - accuracy: 0.8586
Epoch 7/10
2008/2008 [==============================] - 8s 4ms/sample - loss: 0.4718 - accuracy: 0.8760
Epoch 8/10
2008/2008 [==============================] - 8s 4ms/sample - loss: 0.4597 - accuracy: 0.9124
Epoch 9/10
2008/2008 [==============================] - 8s 4ms/sample - loss: 0.6502 - accuracy: 0.8969
Epoch 10/10
2008/2008 [================

#### Save Model/Load Model

In [31]:
nn_model.save('nn_model.h5')

In [5]:
from tensorflow.keras.models import load_model

In [6]:
nn_model = load_model('nn_model.h5')

#### Create Plots

In [8]:
def img_predict(img, nn_model):
    '''
    reads in a single image, proccesses it, and generates predictions from the neural network
    assumes incoming image is square
    returns prediction array
    '''
    
    # Resize to 100x100
    img = cv2.resize(img, (100, 100))
    
    # Reshape to NN friendly shape
    img = img.reshape(1, 100, 100, 3)
    
    # Generate prediction array from given NN model
    pred = nn_model.predict(img.astype(float))
    
    return pred[0]

In [7]:
def plot_pred(nn_model, img, name):

    nn_pred = img_predict(img, nn_model)
    labels = ['cat', 'dog', 'snake', 'penguin', 'dolphin']

    # Plot Image
    fig = plt.figure()

    plt.subplot2grid((3, 2), (0 ,0), rowspan=3)
    plt.imshow(img)
    plt.xticks([])
    plt.yticks([])

    # Plot CNN Prediction
    plt.subplot2grid((3,2), (1, 1))
    plt.bar(list(range(len(nn_pred))), nn_pred, color='firebrick')
    plt.xticks(np.arange(len(nn_pred)), labels)
    plt.ylim([0, 1])
    plt.title('Neural Network')
    plt.yticks([0, .5, 1])

    fig.tight_layout()
    fig.set_size_inches(w=12,h=6)
    
    plt.savefig(name)
    
    plt.close()

In [9]:
full_img = np.array(Image.open('test_images/strip_11.jpg'))

In [14]:
for n in range(0, full_img.shape[0], 36):

    img = full_img[n:612 + n]

    plot_pred(nn_model, img, 'plots/plot_{}.png'.format(str(n).zfill(4)))